# Codes used for training process of Neural Networks: 

## Libraries used for codes before training session: 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import AutoAugment, ColorJitter, Compose

### Calculating mean and standard deviation from training data to normalize training and validation data

In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset_train(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = sorted(os.listdir(root_dir))  # get a list of subdirectory names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # map class names to indices
        self.samples = []
        for target_class in self.classes:
            class_dir = os.path.join(root_dir, target_class)
            if not os.path.isdir(class_dir):
                continue
            for root, _, fnames in sorted(os.walk(class_dir)):
                for fname in sorted(fnames):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        path = os.path.join(root, fname)
                        item = (path, self.class_to_idx[target_class])
                        self.samples.append(item)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # resize the image
            transforms.ToTensor(),  # convert PIL image to a PyTorch tensor
            
            
       #])
            
            
            #ResNet with Landmarks
            transforms.Normalize(mean=[0.4514, 0.3094, 0.2865], 
                          std=[0.1353, 0.1066, 0.0966]),])
            
            #ResNet without Landmarks
       #     transforms.Normalize(mean=[0.4429, 0.3043, 0.2806], 
       #                   std=[0.1187, 0.0874, 0.0728]),])
        
        if transform is not None:
            self.transform = transforms.Compose([self.transform, transform])
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with open(path, 'rb') as f:
            sample = Image.open(f).convert('RGB')
        sample = self.transform(sample)
        return sample, target
    
    def __len__(self):
        return len(self.samples)


In [3]:
train_dataset = CustomDataset_train(root_dir='C:/Users/Bruss/Desktop/Speciale/data/used_data_5/training_data')

In [4]:
train_loader = DataLoader(train_dataset)

In [5]:
# Use a loop to calculate the mean and standard deviation of your training data
def calculate_mean_std(train_loader): 
    mean = 0.
    std = 0.
    for images, _ in train_loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    # Calculate the mean and standard deviation of your training data
    mean /= len(train_loader.dataset)
    std /= len(train_loader.dataset)
    print(f'Mean: {mean}')
    print(f'Standard deviation: {std}')
    return mean, std

In [6]:
calculate_mean_std(train_loader)

Mean: tensor([-0.0627, -0.0475, -0.0609])
Standard deviation: tensor([0.8772, 0.8196, 0.7533])


### Output of calculate_mean_std on train_loader is passed to train dataset and valid dataset. 

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset_train(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = sorted(os.listdir(root_dir))  # get a list of subdirectory names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # map class names to indices
        self.samples = []
        for target_class in self.classes:
            class_dir = os.path.join(root_dir, target_class)
            if not os.path.isdir(class_dir):
                continue
            for root, _, fnames in sorted(os.walk(class_dir)):
                for fname in sorted(fnames):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        path = os.path.join(root, fname)
                        item = (path, self.class_to_idx[target_class])
                        self.samples.append(item)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize of image
            transforms.ToTensor(),  # Tensor converter
            

        
            #mean and std for landmarked data
            transforms.Normalize(mean=[0.4514, 0.3094, 0.2865], 
                           std=[0.1353, 0.1066, 0.0966]),])
            #mean and std for non-landmarked data
        #    transforms.Normalize(mean=[0.4429, 0.3043, 0.2806], 
        #                  std=[0.1187, 0.0874, 0.0728]),])
        
        if transform is not None:
            self.transform = transforms.Compose([self.transform, transform])
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with open(path, 'rb') as f:
            sample = Image.open(f).convert('RGB')
        sample = self.transform(sample)
        return sample, target
    
    def __len__(self):
        return len(self.samples)


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset_valid(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = sorted(os.listdir(root_dir))  # get a list of subdirectory names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # map class names to indices
        self.samples = []
        for target_class in self.classes:
            class_dir = os.path.join(root_dir, target_class)
            if not os.path.isdir(class_dir):
                continue
            for root, _, fnames in sorted(os.walk(class_dir)):
                for fname in sorted(fnames):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        path = os.path.join(root, fname)
                        item = (path, self.class_to_idx[target_class])
                        self.samples.append(item)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # resize the image
            transforms.ToTensor(),  # convert PIL image to a PyTorch tensor
            transforms.Normalize(mean=[0.4514, 0.3094, 0.2865], 
                          std=[0.1353, 0.1066, 0.0966]),])
        
        if transform is not None:
            self.transform = transforms.Compose([self.transform, transform])
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with open(path, 'rb') as f:
            sample = Image.open(f).convert('RGB')
        sample = self.transform(sample)
        return sample, target
    
    def __len__(self):
        return len(self.samples)


In [ ]:
train_dataset = CustomDataset_train(root_dir='C:/Users/Bruss/Desktop/Speciale/data/used_data_landmarks/training_data')
validation_data = CustomDataset_valid(root_dir='C:/Users/Bruss/Desktop/Speciale/data/used_data_landmarks/testing_data')

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=26, shuffle=True)

val_loader = DataLoader(validation_data, batch_size=26, shuffle = False)